## WEB Scrapping Using BeautifulSoup webdriver 

In [10]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# URL de la page Web que vous souhaitez extraire
url = 'https://www.kooora.com/?n=0&o=n'

# Utiliser Selenium WebDriver pour simuler un navigateur
driver = webdriver.Chrome()  # Vous devez avoir installé le WebDriver de Chrome
driver.get(url)

# Attendre que la page soit complètement chargée
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.TAG_NAME, "div")))

# Obtenir le contenu HTML complètement rendu
html = driver.page_source

# Fermer le WebDriver
driver.quit()

# Analyser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')




In [11]:

# Trouver tous les divs avec la classe "newsList"
divs = soup.findAll('div', attrs={'class': "newsList"})

# Initialisation d'une liste pour stocker les titres
titles = []
liens= []
# Boucle pour parcourir chaque div
for div in divs:
    # Trouver tous les éléments li dans le div courant
    li_elements = div.findAll('li')

    # Pour chaque élément li, trouver tous les éléments strong et récupérer leur contenu textuel
    for li in li_elements:
        strong_elements = li.findAll('strong')
        a_elements=li.findAll('a')
        for strong,p in zip(strong_elements,a_elements):
            # Ajouter le contenu textuel du strong à la liste des titres
            titles.append(strong.text)
            liens.append(p.get('href'))

print(titles)
print(liens)

['الخالدية يتصدر الدوري البحريني مؤقتا', 'أحمد حمدي: جمهور الزمالك وفي وعظيم.. وهدفنا لقب الكونفدرالية', 'محمد شحاتة: جمهور الزمالك لا يدعمني بمفردي', 'استبعاد معلول وتاو من قائمة الأهلي ضد إنبي', 'مدرب فيوتشر يوجه طلبا للكاف بعد مغادرة الكونفيدرالية', 'صورة.. رحيمي يفقد وعيه في لقاء العين والنصر', 'بوكيتينو: لاعبو تشيلسي غير ناضجين', 'بعد التأهل الثالث.. ماذا قدم الزمالك في المربع الذهبي للكونفيدرالية؟', 'بالصور: الزمالك يعبر فيوتشر إلى نصف نهائي الكونفدرالية', 'هيثم فاروق: الزمالك يلعب دون مدرب', 'روي كين: أداء اليونايتد يليق بفرق وسط الجدول', 'جاياردو يستبعد ثلاثي اتحاد جدة', 'زيدان: نجم الريال استثنائي.. ويعجبني بشدة', 'حمدي يضع بصمته الأولى مع الزمالك', 'هل يكرس جيسوس عقدة كاسترو؟']
['?n=1319172&pg=1&o=n', '?n=1319169&pg=1&o=n', '?n=1319171&pg=1&o=n', '?n=1319168&pg=1&o=n', '?n=1319170&pg=1&o=n', '?n=1319166&pg=1&o=n', '?n=1319158&pg=1&o=n', '?n=1319165&pg=1&o=n', '?n=1319164&pg=1&o=n', '?n=1319167&pg=1&o=n', '?n=1319160&pg=1&o=n', '?n=1319162&pg=1&o=n', '?n=1319062&pg=1&o=n', '?n

In [12]:
import requests
from bs4 import BeautifulSoup

contents = []
for i in liens:
    url = 'https://www.kooora.com/' + i

    # Utiliser Selenium WebDriver pour simuler un navigateur
    driver = webdriver.Chrome()  # Vous devez avoir installé le WebDriver de Chrome
    driver.get(url)

    # Attendre que la page soit complètement chargée
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of_element_located((By.TAG_NAME, "div")))

    # Obtenir le contenu HTML complètement rendu
    html = driver.page_source

    # Fermer le WebDriver
    driver.quit()
    soup = BeautifulSoup(html, 'html.parser')

    # Rechercher des balises <div> avec une classe spécifique
    div_content = soup.findAll('div',attrs={'class':'articleBody'})
    contents.append(div_content)



In [13]:
articles = [content[0].text for content in contents]


In [18]:
json_content = [{'title':title , 'article' :article } for title, article in zip(titles,articles)]

## Store the content in MongoDB

In [24]:
import pymongo
import json

client = pymongo.MongoClient("mongodb://localhost:27017/")

# Sélectionner une base de données
db = client["demoDB"]

# Sélectionner une collection
collection = db["sport"]

# Insérer le contenu JSON dans la collection
collection.insert_many(json_content)

print("Contenu JSON inséré avec succès dans MongoDB.")

Contenu JSON inséré avec succès dans MongoDB.


## pipeline 

In [6]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer


In [7]:
def clean_str(text):

#remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,'', text)
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    text = text.replace('و','وو')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    text = text.replace('أ', 'ا')
    text = text.replace('إ', 'ا')
    text = text.replace('آ', 'ا')
    text = text.replace('ى', 'ي')
    return text

def tokenize_text(text):
    # Tokenization du texte
    tokens = word_tokenize(text)
    return tokens

def remove_stopwords(tokens):
    # Supprimer les mots vides
    stop_words = set(stopwords.words('arabic'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens


# Fonction pour normaliser les données
def normalize_text(text):
    text = clean_str(text)
    tokens = tokenize_text(text)
    tokens = remove_stopwords(tokens)
    return ' '.join(tokens)

# Fonction pour discrétiser les données
def discretize_text(text):
    vectorizer = CountVectorizer()
    vectorizer.fit_transform([text])
    vocabulary = vectorizer.get_feature_names_out()
    return vocabulary

def pineLine(text):
    text = normalize_text(text)
    text = discretize_text(text)
    return text

## Import the contents from MongoDB

In [8]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")

# Sélectionner une base de données
db = client["demoDB"]

# Sélectionner une collection
collection = db["sport"]

# Récupérer tous les documents de la collection
documents = collection.find()

# Stocker les documents dans une liste de dictionnaires
contents=[document for document in documents]

In [9]:
contents_cleaning = [{'title':item['title'],'article':pineLine(item['article'])} for item in contents  ]

## stemming & lemmatization 



Stemming and lemmatization are both techniques used in natural language processing (NLP) to reduce words to their base forms. However, they differ in their approach and the resulting output:

### Stemming:

- Simpler and faster approach.
- Removes suffixes from words using predefined rules.
- Results in a base form that might not be a real word (e.g., "running" stemmed to "runn").
- Less accurate but computationally efficient.

### Lemmatization:

- More complex and slower approach.
- Uses a dictionary and morphological analysis to identify the actual base word (lemma).
- Results in a valid word (lemma) that represents the morphological root (e.g., "running" lemmatized to "run").
- More accurate but requires more processing power.

In [10]:
from nltk.stem.isri import ISRIStemmer
def light_stem(contents_cleaning):
    contents_stemming=[]
    stemmer = ISRIStemmer()
    for item in  contents_cleaning:
        item_stem=[stemmer.stem(token) for token in item['article']]
        contents_stemming.append({"title":item['title'],'article':item_stem})
    return contents_stemming

contents_stemming=light_stem(contents_cleaning)
contents_cleaning[0]['article'][10:20]

array(['اخري', 'اقيمت', 'ال', 'الاحد', 'الاخير', 'الاهداف', 'الاوول',
       'البحرين', 'البحريني', 'البسيتين'], dtype=object)

In [32]:
import qalsadi.lemmatizer

def light_lemm(contents_cleaning):
    contents_lemm=[]
    lemmer = qalsadi.lemmatizer.Lemmatizer()
    for item in  contents_cleaning:
        item_stem=[lemmer.lemmatize(token) for token in item['article']]
        contents_lemm.append({"title":item['title'],'article':item_stem})
    return contents_stemming
contents_lemm=light_lemm(contents_cleaning)

In [33]:
contents_lemm[0]['article'][10:20]

['اخر', 'اقم', 'ال', 'احد', 'اخر', 'هدف', 'اول', 'بحر', 'بحر', 'بسي']

## NER Method

In [42]:
from deep_translator import GoogleTranslator

In [39]:
contents_cleaning[0]['article']

array(['11', '15', '17', '19', '22', '30', '31', '45', '86', 'احمد',
       'اخري', 'اقيمت', 'ال', 'الاحد', 'الاخير', 'الاهداف', 'الاوول',
       'البحرين', 'البحريني', 'البسيتين', 'التاسع', 'الترتيب', 'الثامن',
       'الثاني', 'الجوولة', 'الحالة', 'الحد', 'الخالدية', 'الخسارة',
       'الرابع', 'الرفاع', 'الرياضية', 'السابع', 'السادس', 'الشباب',
       'الشرقاووي', 'الشرقي', 'الشيخ', 'العاشر', 'الفووز', 'الفيس',
       'اللقاء', 'المباراة', 'المتذيل', 'المركز', 'المركزين', 'المسابقة',
       'الممتاز', 'المنامة', 'النجمة', 'النقطة', 'الهدف', 'الووحيد',
       'الووطني', 'الي', 'امير', 'ان', 'بالخطا', 'بالدقيقة', 'بجدوول',
       'بفارق', 'بن', 'بنتيجة', 'بهدف', 'بهذا', 'ترتيب', 'تووباريس',
       'تووقف', 'جاء', 'جانبه', 'جمعتهما', 'حمد', 'خلال', 'خليفة',
       'دقائق', 'دووري', 'رصيد', 'رصيده', 'رفع', 'رووستائي', 'ستاد',
       'سترة', 'سجل', 'سجله', 'صاحب', 'صاحبي', 'صدارة', 'طريق', 'علي',
       'فاز', 'فاسيووندوو', 'فرانسيسكوو', 'فلامرزي', 'في', 'لاعب',
       'لاعبه', 'لجدوول',

In [83]:
import nltk
nltk.download('words')
sentence = "مرحبا بالعالم!"
from nltk import word_tokenize, pos_tag, ne_chunk
traitement = pos_tag([GoogleTranslator(source='auto', target='en').translate(item) for item in contents_cleaning[0]['article']])
traitement=resultat = [x[1] for x in traitement]
pos_rules = [(mot,role) for mot , role in zip(contents_cleaning[0]['article'],traitement)]
pos_rules



[nltk_data] Downloading package words to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


[('11', 'CD'),
 ('15', 'CD'),
 ('17', 'CD'),
 ('19', 'CD'),
 ('22', 'CD'),
 ('30', 'CD'),
 ('31', 'CD'),
 ('45', 'CD'),
 ('86', 'CD'),
 ('احمد', 'NNP'),
 ('اخري', 'DT'),
 ('اقيمت', 'VBD'),
 ('ال', 'DT'),
 ('الاحد', 'NNP'),
 ('الاخير', 'NN'),
 ('الاهداف', 'NNP'),
 ('الاوول', 'NNP'),
 ('البحرين', 'VBD'),
 ('البحريني', 'NNP'),
 ('البسيتين', 'NNP'),
 ('التاسع', 'NNP'),
 ('الترتيب', 'NNP'),
 ('الثامن', 'NNP'),
 ('الثاني', 'NN'),
 ('الجوولة', 'NNP'),
 ('الحالة', 'NN'),
 ('الحد', 'NN'),
 ('الخالدية', 'NNP'),
 ('الخسارة', 'NN'),
 ('الرابع', 'NN'),
 ('الرفاع', 'NNP'),
 ('الرياضية', 'NNP'),
 ('السابع', 'NNP'),
 ('السادس', 'NNP'),
 ('الشباب', 'NNP'),
 ('الشرقاووي', 'NNP'),
 ('الشرقي', 'NNP'),
 ('الشيخ', 'NNP'),
 ('العاشر', 'NNP'),
 ('الفووز', 'NNP'),
 ('الفيس', 'NNP'),
 ('اللقاء', 'NNP'),
 ('المباراة', 'VB'),
 ('المتذيل', 'NN'),
 ('المركز', 'NNP'),
 ('المركزين', 'NNP'),
 ('المسابقة', 'NNP'),
 ('الممتاز', 'NNP'),
 ('المنامة', 'NNP'),
 ('النجمة', 'NN'),
 ('النقطة', 'NN'),
 ('الهدف', 'NN'),
 ('الووح

In [18]:
import spacy

# Charger le modèle pour l'arabe
nlp = spacy.load("xx_ent_wiki_sm")

texte_arabe=contents[0]['article']
print(texte_arabe)

# Traiter le texte avec spaCy
doc = nlp(texte_arabe)

# Afficher les entités nommées
for ent in doc.ents:
    print(ent.text, ent.label_)


فاز الخالدية على البسيتين (1-0)، مساء الأحد، على ستاد البحرين الوطني، في الجولة 15 من دوري ناصر بن حمد البحريني الممتاز.جاء هدف اللقاء الوحيد بعد مرور 7 دقائق عن طريق لاعب الخالدية علي الشرقاوي.ورفع الخالدية رصيده بهذا الفوز إلى 30 نقطة في صدارة مؤقتة لجدول ترتيب المسابقة بفارق نقطتين عن الرفاع صاحب المركز الثاني.وتوقف رصيد البسيتين عند النقطة 15 في المركز التاسع بفارق الأهداف عن الرفاع الشرقي صاحب المركز العاشر وبنقطة عن النجمة صاحب المركز الثامن.في مباراة أخرى فاز المنامة بنتيجة (2-1) على سترة خلال المباراة التي أُقيمت على ستاد مدينة خليفة الرياضية.وتقدم المنامة بعد مرور 5 دقائق بهدف سجله لاعبه ألفيس مارلي قبل أن يتعادل أمير روستائي لفريق سترة بالدقيقة 8، ثم سجل فاسيوندو توباريس الهدف الثاني للمنامة بالدقيقة 45.ورفع المنامة رصيده بهذا الفوز إلى النقطة 22 في المركز الرابع بجدول الترتيب، فيما توقف رصيد سترة عند 17 نقطة في المركز السابع.من جانبه فاز الحد بنتيجة (2-0) على الحالة خلال المباراة التي جمعتهما على ستاد الشيخ علي بن محمد آل خليفة.جاء الهدف الأول للحد عن طريق لاعب الحالة فرانسي

Using spaCy for processing Arabic text, especially for tasks like Named Entity Recognition (NER), presents several challenges due to the unique characteristics of the Arabic language. Here are some of the key issues:
- Limited Language Resources: Compared to languages like English, Arabic has fewer available language resources, such as annotated corpora, pre-trained models, and linguistic tools. This scarcity of resources can hinder the development and performance of NER systems for Arabic.
- Named Entity Variation: Named entities in Arabic can vary significantly in form, especially when considering the various ways in which names can be represented (e.g., full names, titles, honorifics). Additionally, Arabic named entities often include multi-word expressions, which can further complicate detection and classification.